# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

## Basic FABRIC Slice Configuration

In [1]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         4  125/128  1016/1024  55750/55800    253/254                 2/2                             0/0                            6/6                 0/0               3/3
MAX         10  248/320  2326/2560  115058/116400  579/635                 2/2                             4/4                            16/16               4/4               6/6
TACC        10  320/320  2560/2560  116400/116400  635/635                 2/2                             4/4                            16/16               4/4               6/6
MICH         6  192/192  1536/1536  60600/60600    381/381                 2/2

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [3]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv6Network, IPv4Network

# Slice 
slice_name = 'VCCchernovik'

# [site1,site2,site3] = fablib.get_random_sites(count=3)
site1 = site2 = site3 = 'MAX'
print(f"Sites: {site1},{site2},{site3}")


# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 100

# Hosts
h1_name = "h1"
h11_name = "h11"
h2_name = "h2"
h22_name = "h22"


# h1_subnet=IPv4Network('10.0.0.0/16')
# h1_addr=IPv4Address('10.0.1.1')
# h11_addr=IPv4Address('10.0.1.11')

# h2_subnet=h1_subnet #IPv4Network('10.0.0.0/16')
# h2_addr=IPv4Address('10.0.2.2')
# h22_addr=IPv4Address('10.0.2.22')

host_cores = 2
host_ram = 8
host_disk = 20

net_h1_name = 'net_h1'
net_h11_name = 'net_h11'
net_h2_name = 'net_h2'
net_h22_name = 'net_h22'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

s1_iface_h1,s1_iface_h11,s1_iface_to_s2,s1_iface_to_s3 = "s1_iface_h1","s1_iface_h11","s1_iface_to_s2","s1_iface_to_s3"  
s2_iface_h2,s2_iface_h22,s2_iface_to_s1,s2_iface_to_s3 = "s2_iface_h2","s2_iface_h22","s2_iface_to_s1","s2_iface_to_s3" 
s3_iface_to_s1,s3_iface_to_s2 = "s3_iface_to_s1","s3_iface_to_s2"

# All node properties
image = 'default_ubuntu_20'

Sites: MAX,MAX,MAX


In [4]:
# slice = fablib.get_slice(name=slice_name)

### Create the Slice

In [5]:
import datetime
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    def addSwitchNIC(nodeName,site,ifaceNames):
        node = slice.add_node(name=nodeName, site=site,  image=image, cores=switch_cores, ram=switch_ram, disk=switch_disk)
        ifaceObj = {}
        for name in ifaceNames:
            iface =  node.add_component(model='NIC_Basic', name=name).get_interfaces()[0]
            ifaceObj[name] = iface
        return node, ifaceObj
    
    # Add switch node s1
    s1, s1_ifaces = addSwitchNIC(s1_name,site1,[s1_iface_h1,s1_iface_h11,s1_iface_to_s2,s1_iface_to_s3])

    # Add switch node s2
    s2, s2_ifaces = addSwitchNIC(s2_name,site2,[s2_iface_h2,s2_iface_h22,s2_iface_to_s1,s2_iface_to_s3])
    
    # Add switch node s3
    s3, s3_ifaces = addSwitchNIC(s3_name,site3,[s3_iface_to_s1,s3_iface_to_s2])
    
    def addHostNIC(nodeName,componentName,site):
        node = slice.add_node(name=nodeName, site=site, image=image, cores=host_cores, ram=host_ram, disk=host_disk)
        iface = node.add_component(model='NIC_Basic', name=componentName).get_interfaces()[0]
        return node,iface
    
    # Add host node h1
    h1,h1_iface = addHostNIC(h1_name,"h1_nic",site1)

    # Add host node h11
    h11 , h11_iface = addHostNIC(h11_name,"h11_nic",site1)
    
    # Add host node h2
    h2, h2_iface = addHostNIC(h2_name,"h2_nic",site2)
    
    # Add host node h3
    h22, h22_iface = addHostNIC(h22_name,"h22_nic",site2)
    
    #Add swtich networks
    switch_net1 = slice.add_l3network(name=net_s1_s2_name, interfaces=[s1_ifaces[s1_iface_to_s2], s2_ifaces[s2_iface_to_s1]], type='IPv4')
    swtich_net2 = slice.add_l3network(name=net_s2_s3_name, interfaces=[s2_ifaces[s2_iface_to_s3], s3_ifaces[s3_iface_to_s2]], type='IPv4')
    swtich_net3 = slice.add_l3network(name=net_s1_s3_name, interfaces=[s3_ifaces[s3_iface_to_s1], s1_ifaces[s1_iface_to_s3]], type='IPv4')

    #Add host networks 
    host_net1 = slice.add_l3network(name=net_h1_name, interfaces=[s1_ifaces[s1_iface_h1], h1_iface], type='IPv4')
    host_net11 = slice.add_l3network(name=net_h11_name, interfaces=[s1_ifaces[s1_iface_h11], h11_iface], type='IPv4')
    host_net2 = slice.add_l3network(name=net_h2_name, interfaces=[s2_ifaces[s2_iface_h2], h2_iface], type='IPv4')
    host_net22 = slice.add_l3network(name=net_h22_name, interfaces=[s2_ifaces[s2_iface_h22], h22_iface], type='IPv4')
    
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


-----------  ------------------------------------
Slice Name   VCCchernovik
Slice ID     e26440b3-80e2-4202-b6da-39e767469f65
Slice State  StableOK
Lease End    2022-09-05 07:55:50 +0000
-----------  ------------------------------------

Retry: 17, Time: 237 sec

ID                                    Name    Site    Host                         Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  -------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
beac1d2d-ef12-4e19-b31b-9fd3bb23eb03  s1      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  63.239.135.72    Active
5fb2fe60-5743-49c2-8df4-ab58658f2159  s2      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  63.239.135.89    Active
da464564-7091-4a7d-b7ef-c3cd3c23804f  s3      MAX     max-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  6

In [25]:
# network1 = slice.get_network(name=network1_name)
#     switch_net1 = slice.get_network(name=net_s1_s2_name, interfaces=[s1_ifaces[s1_iface_to_s2], s2_ifaces[s2_iface_to_s1]], type='IPv4')
#     swtich_net2 = slice.get_network(name=net_s2_s3_name, interfaces=[s2_ifaces[s2_iface_to_s3], s3_ifaces[s3_iface_to_s2]], type='IPv4')
#     swtich_net3 = slice.get_network(name=net_s1_s3_name, interfaces=[s3_ifaces[s3_iface_to_s1], s1_ifaces[s1_iface_to_s3]], type='IPv4')

#     #Add host networks 
#     host_net1 = slice.get_network(name=net_h1_name, interfaces=[s1_ifaces[s1_iface_h1], h1_iface], type='IPv4')
#     host_net11 = slice.get_network(name=net_h11_name, interfaces=[s1_ifaces[s1_iface_h11], h11_iface], type='IPv4')
#     host_net2 = slice.get_network(name=net_h2_name, interfaces=[s2_ifaces[s2_iface_h2], h2_iface], type='IPv4')
#     host_net22 = slice.get_network(name=net_h22_name, interfaces=[s2_ifaces[s2_iface_h22], h22_iface], type='IPv4')

NameError: name 'network1_name' is not defined

In [6]:
print(slice.get_network(name=net_h1_name))


-----------------  ------------------------------------
ID                 d50c0f45-24df-44af-a735-4097fc71e26a
Name               net_h1
Layer              L3
Type               FABNetv4
Site               MAX
Gateway            10.130.4.1
L3 Subnet          10.130.4.0/24
Reservation State  Active
Error Message
-----------------  ------------------------------------


In [7]:
config_threads = {}

In [8]:
def checkInetConnection(node):
    try:
        #If the node is an IPv4 Node then configure NAT64
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            print("This code only works with ip4, reconfigure nodes")
            return

        #Access non-IPv4 Services
        config_thread = node.execute_thread(f'sudo apt-get update -qq && sudo apt install net-tools && git clone https://github.com/fabric-testbed/jupyter-examples.git && ls jupyter-examples')
        config_threads[node]=config_thread

    except Exception as e:
        print(f"Exception: {e}")

In [9]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        checkInetConnection(node)
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [ ]:
def printThreadOutputs(con_threads):
    try: 
        for node, thread in con_threads.items():
            stdout, stderr = thread.result()
            print(f"Config thread node {node.get_name()} complete")
            # checkIP(node)
    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc() 
printThreadOutputs(config_threads)
config_threads = {}    

### Print Details

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("----")
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        # print(f"   Cores             : {node.get_cores()}")
        # print(f"   RAM               : {node.get_ram()}")
        # print(f"   Disk              : {node.get_disk()}")
        # print(f"   Image             : {node.get_image()}")
        # print(f"   Image Type        : {node.get_image_type()}")
        # print(f"   Host              : {node.get_host()}")
        # print(f"   Site              : {node.get_site()}")
        # print(f"   Management IP     : {node.get_management_ip()}")
        # print(f"   Reservation ID    : {node.get_reservation_id()}")
        # print(f"   Reservation State : {node.get_reservation_state()}")
        ssh = node.get_ssh_command().replace("ssh -i /home/fabric/work/fabric_config/.ssh/id_rsa -J durbek_gafurov_0000000854@bastion-1.fabric-testbed.net","./connect.sh")
        print(f"   SSH Command       : {ssh}")
        # print(f"   Components        :  ")
        # for component in node.get_components():
        #     print(f"      Name             : {component.get_name()}")
        #     print(f"      Details          : {component.get_details()}")
        #     print(f"      Disk (G)         : {component.get_disk()}")
        #     print(f"      Units            : {component.get_unit()}")
        #     print(f"      PCI Address      : {component.get_pci_addr()}")
        #     print(f"      Model            : {component.get_model()}")
        #     print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ", end = " ")
        for interface in node.get_interfaces():
        #     print(f"       Name                : {interface.get_name()}")
        #     print(f"           Bandwidth           : {interface.get_bandwidth()}")
            # print(f"           VLAN                : {interface.get_vlan()}")  
            # print(f"           MAC                 : {interface.get_mac()}") 
            print(interface.get_os_interface(), end =" ")
    # for network in slice.get_l2networks():
    #     print("Network:")
    #     print(f"    Name:            {network.get_name()}")
    # print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Configure Nodes


In [ ]:
config_threads = {}

In [ ]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/" 
def addIP(name,net_name,node_addr,node_subnet):
    node = slice.get_node(name=name)        
    h1_os_iface = node.get_interface(network_name=net_name).ip_addr_add(addr=node_addr, subnet=node_subnet)
    config_thread = node.execute_thread(host_config_script)
    config_threads[node]=config_thread
    return node
    
try:
    
    h1 = addIP(h1_name,net_h1_name,h1_addr,h1_subnet)
    # print(h1.execute(f'sudo arp -i ens7 -s 10.0.1.0 08:00:00:00:01:00 && arp -a'))
    h11 = addIP(h11_name,net_h11_name,h11_addr,h1_subnet)
    # print(h11.execute(f'sudo arp -i ens7 -s 10.0.1.10 08:00:00:00:01:00 && arp -a'))
    h2 = addIP(h2_name,net_h2_name,h2_addr,h2_subnet)
    # print(h2.execute(f'sudo arp -i ens7 -s 10.0.2.3 08:00:00:00:02:00 && arp -a'))
    h22 = addIP(h22_name,net_h22_name,h22_addr,h2_subnet)
    # print(h22.execute(f'sudo arp -i ens7 -s 10.0.2.23 08:00:00:00:02:00 && arp -a'))


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [ ]:
# def checkIP(node):
#     try:        
#         stdout, stderr = node.execute(f'ifconfig | grep "inet 10.0."')
#         print(stdout)
#         print(stderr)
#     except Exception as e:
#         print(f"Exception: {e}")

In [ ]:
printThreadOutputs(config_threads)
config_threads = {}

## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



Below are commands to let sudo work with the global proxy.

In [ ]:
print(slice.get_node(name=s1_name).get_interface(network_name=net_s1_s2_name))
print(h1.get_interface(network_name=net_h1_name))

In [ ]:
h1Mac = h1.get_interface(network_name=net_h1_name).get_mac()
h11Mac = h11.get_interface(network_name=net_h11_name).get_mac()
h2Mac = h2.get_interface(network_name=net_h2_name).get_mac()
h22Mac = h22.get_interface(network_name=net_h22_name).get_mac()

s1_net_s1_s2_Mac = slice.get_node(name=s1_name).get_interface(network_name=net_s1_s2_name).get_mac()
s2_net_s1_s2_Mac = slice.get_node(name=s2_name).get_interface(network_name=net_s1_s2_name).get_mac()

s1_net_s1_s3_Mac = slice.get_node(name=s1_name).get_interface(network_name=net_s1_s3_name).get_mac()
s3_net_s1_s3_Mac = slice.get_node(name=s3_name).get_interface(network_name=net_s1_s3_name).get_mac()

s2_net_s2_s3_Mac = slice.get_node(name=s2_name).get_interface(network_name=net_s2_s3_name).get_mac()
s3_net_s2_s3_Mac = slice.get_node(name=s3_name).get_interface(network_name=net_s2_s3_name).get_mac()

print(h1Mac,h11Mac,h2Mac,h22Mac)
def printToFile(text,filename):
    text_file = open(filename, "w")
    n = text_file.write(text)
    text_file.close()
s1_command = """table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.1/32 => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.11/32 => {} 2
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.2/32 => {} 3
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.22/32 => {} 4
""".format(h1Mac, h11Mac,s2_net_s1_s2_Mac,s3_net_s1_s3_Mac)
s2_command = """table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 2
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.1/32 => {} 3
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.11/32 => {} 4
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.2/32 => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.22/32 => {} 2
""".format(s1_net_s1_s2_Mac,s3_net_s2_s3_Mac,h2Mac,h22Mac)
s3_command = """table_clear MyEgress.swtrace
table_clear MyIngress.ipv4_lpm 
table_set_default MyEgress.swtrace add_swtrace 3
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.0/24 => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.0/24 => {} 2
""".format(s1_net_s1_s3_Mac,s2_net_s2_s3_Mac)
printToFile(s1_command, "./scripts/s1command.txt")
printToFile(s2_command, "./scripts/s2command.txt")
printToFile(s3_command, "./scripts/s3command.txt")
print(s1_command, "./scripts/s1command.txt")
print(s2_command, "./scripts/s2command.txt")
print(s3_command, "./scripts/s3command.txt")

In [ ]:
def configureSwitch(name,network_names):
    try:
        node = slice.get_node(name=name)
        os_ifaces = ""
        for net_name in network_names:
            iface = node.get_interface(network_name=net_name).get_os_interface()
            if iface is None:
                print("iface is none")
                return
            os_ifaces=os_ifaces+iface+" "

        file_attributes = node.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
        command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {os_ifaces}  > /tmp/script.log 2>&1' && tail /tmp/script.log"
        config_threads[node] = node.execute_thread(command)
        print(command)


    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()

In [ ]:
configureSwitch(s1_name,[net_h1_name,net_h11_name,net_s1_s2_name,net_s1_s3_name]) 
configureSwitch(s2_name,[net_h2_name,net_h22_name,net_s1_s2_name,net_s2_s3_name])
configureSwitch(s3_name,[net_s1_s3_name,net_s2_s3_name])

In [ ]:
printThreadOutputs(config_threads)
config_threads = {}

### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [ ]:

for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}command.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [ ]:
h1.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h1.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h2.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h2.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h11.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h11.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')
h22.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h22.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

In [ ]:
print(h1)

## H1 sends packet, H2 recieves

In [ ]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.2.2 "message100" 50\'')
print(h2.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

## H1 sends packet, H11 recieves

In [ ]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.1.11 "message100" 50\'')
print(h11.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

## H2 sends packet, H1 recieves

In [ ]:
h2.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.1.1 "message100" 50\'')
print(h1.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

## H22 sends packet, H11 recieves

In [ ]:

h22.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.1.11 "message100" 50\'')
print(h11.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

## Delete Slice

In [ ]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Fail: {e}")
#     traceback.print_exc()